Hello, let me explained this chatbot. 

Why i'm rename Simple Chatbot? because this ChatBot just only using Clasification Concept. We have list of input text conversation and output conversation. This chatbot using data history in dataset, so if this Chatbot looking sentences not in training, it cann't detection. I'm not sure this chatbot can be responses using new data from user, but if we want use like that, we can using Generate Chatbot. This my argument has I know, but if you have more knowladge, let's discussion soon. 

In [22]:
#load data
import json
import pandas as pd

with open('src/humanText.json') as data_file:
    data = json.load(data_file)

In [51]:
textInput = []
intents = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        textInput.append(pattern)
        intents.append(intent['tag'])

df = pd.DataFrame({'textInput': textInput,
                    'intents': intents})
df.head()

,textInput,intents
0,Hai,salam
1,Hi,salam
2,Halo,salam
3,Apa Kabar,salam
4,Selamat Pagi,salam


In [52]:
df.intents.value_counts()

salam                 10
bye                    8
komunitas              7
nama                   6
pekerjaan              5
ngodingpython          5
ngodingpython_typo     5
youtube                5
kemampuan              5
Name: intents, dtype: int64

# Pre Processing

In [53]:
# data cleansing
import string

# convert lowercase
df.textInput = df.textInput.apply(lambda x: x.lower())

# remove punctuation
exclude = set(string.punctuation)
df.textInput = df.textInput.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

### Label Encoding to labelling tag category

In [29]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

le = LabelEncoder()
y_train = le.fit_transform(df.intents)
y_train = to_categorical(y_train)

In [54]:
all_vocab = []
length = []

for idx, row in df.iterrows():
    sent = row['textInput']
    [all_vocab.append(i) for i in sent.split()]
    length.append(len(sent.split()))
#len(all_vocab)
#max(length)
#len(set(all_vocab))

### Vectorization to convert sentences be vector

In [55]:
from tensorflow.keras.layers import TextVectorization

max_vocab_length = 86
max_length = 6

text_vectorization = TextVectorization(max_tokens=max_vocab_length,
                                       standardize='lower_and_strip_punctuation',
                                       split='whitespace',
                                       ngrams=None,
                                       output_mode='int',
                                       output_sequence_length=max_length
                                       )

In [57]:
text_vectorization.adapt(df.textInput)
#text_vectorization.get_vocabulary()
#text_vectorization('halo ara ingin membantu')

### Embedding to create arrary from sentences

In [ ]:
from tensorflow.keras.layers import Embedding
embedding = Embedding(input_dim=max_vocab_length,
                      output_dim=16,
                      embeddings_initializer="uniform",
                      input_length=max_length)
import numpy as np
res_embed = embedding(np.array([[71, 17,  7, 13,  0,  0]]))
res_embed

# Modelling

### Neural Network

### LSTM

In [61]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, LSTM
inputs = Input(shape=(1,), dtype='string')
x = text_vectorization(inputs)
x = embedding(x)
x = LSTM(12)(x)
outputs = Dense(9, activation='softmax')(x)
model_lstm = Model(inputs, outputs, name="LSTM_model")

In [62]:
# compile model
model_lstm.compile(loss='categorical_crossentropy',
                   optimizer='adam',
                   metrics=["accuracy"])

In [63]:
model_lstm.fit(df.textInput,
               y_train,
               epochs=150,
               verbose=0)

In [65]:
model_lstm.evaluate(df.textInput, y_train)

2/2 [==============================] - 2s 0s/step - loss: 0.1423 - accuracy: 1.0000


[0.1422586739063263, 1.0]

In [66]:
model_lstm.save("bot_model.tf")

INFO:tensorflow:Assets written to: bot_model.tf\assets


INFO:tensorflow:Assets written to: bot_model.tf\assets


In [67]:
import pickle
le_filename = open("label_encoder.pickle", "wb")
pickle.dump(le, le_filename)
le_filename.close()

### Neural Network